In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
df_essay=pd.read_csv('/home/semipro321/OneDrive/School/McGill/Research Projects/Essay/Data/essay11/raw_all_essays.csv')

In [3]:
df_essay['cleaned_text']

0        Iam 25 years old. YEsterday I got a car for my...
1        I am 25 years old I am Growing up and I am sta...
2        I would be married. I would be a school teache...
3        I am 25 years old and I am training to be a ve...
4        I am an air hostess. I go to interesting place...
                               ...                        
10504    I am just starting a new job and I work eight ...
10505    I have a wife a four children I have a good Jo...
10506    Helo my name is [name] I am 25 years of age I ...
10507    I am 25 years of age, my job is working in a b...
10508    "I am now 25 years old I am a cartoonist for t...
Name: cleaned_text, Length: 10509, dtype: object

In [4]:
df_sample = df_essay.sample(10)

In [5]:
import language_tool_python
tool = language_tool_python.LanguageTool('en-GB')  

In [11]:
text = "When I am 25 chemistry is important to me because it is what I am going to do. (I hope) I would live in a semi detached house with three bed-rooms. I hope to have a family and only two children. I would leave my children in the care of a redistrad child minder while I go to work. I will learn to drive and have a car. (a Estate car if possible) I would come home from work at half past 4 and get tea ready. I would pay the child minder 3 a week. (or 3005 because of the decimal system) When my children go to school I would help them a little bit (not to much) with they home work at weekends we shall go down to the forset. (on Saturday_ and have picnics. For birthday treats I will let my children pick were they want to go I would let there imagenation ran free and won't laugh at them if they belive in fairys I will let the pick there own holidays. (as long as there not to expensiv BUT I wouldnt let them run to free. I would care and look after them untill they are to old to be looked after when they have a family I would help them all I can I would'nt fuss to much at all. and I'd let them had a wonderfull and exicinting life."

In [12]:
tool.correct(text)

"When I am 25 chemistry is important to me because it is what I am going to do. (I hope) I would live in a semi-detached house with three bed-rooms. I hope to have a family and only two children. I would leave my children in the care of a registrar child minder while I go to work. Furthermore, I will learn to drive and have a car. (an Estate car if possible) I would come home from work at half past 4 and get tea ready. I would pay the child minder 3 a week. (or 3005 because of the decimal system) When my children go to school I would help them a little bit (not too much) with they homework at weekends we shall go down to the forest. (on Saturday_ and have picnics. For birthday treats I will let my children pick were they want to go I would let there imagination ran free and won't laugh at them if they believe in fairies I will let the pick their own holidays. (as long as there not to expensive BUT I wouldn't let them run to free. I would care and look after them until they are too old 

In [22]:
import spacy
import contextualSpellCheck
import concurrent.futures


# Load the spaCy model and add the contextual spell checker
nlp = spacy.load('en_core_web_md')
nlp.max_length = 1500000  # Set this to a value that fits your needs
contextualSpellCheck.add_to_pipe(nlp)
# Function to split text into chunks (e.g., sentences)
def split_text_into_chunks(text, chunk_size=1000):
    sentences = text.split('.')
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) > chunk_size:
            chunks.append(current_chunk)
            current_chunk = sentence + '.'
        else:
            current_chunk += sentence + '.'
    chunks.append(current_chunk)
    return chunks

# Process each chunk and handle the spell check outcome
def process_chunk(chunk):
    doc = nlp(chunk)
    # Check if spell check was performed and return the corrected text
    return doc._.outcome_spellCheck if doc._.performed_spellCheck else chunk

# Process large text in parallel
def process_large_text_parallel(text):
    chunks = split_text_into_chunks(text)
    corrected_chunks = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Map each chunk to process_chunk and collect results
        results = executor.map(process_chunk, chunks)
        # Collect the corrected chunks
        corrected_chunks = list(results)
    return ' '.join(corrected_chunks)

# Example essay text

essay_text = "I moserm i my 25 years old ken we all the pisinmy sek a i I was. PJ in the home in the kisn 3 Perws in my home I get upin the mon en I get the Bets in the mon en I get my hsp up of wek at sckot My house is wait for ne 20 en I mak a tea of me serm mif I at a Sikaimukusto I was up in my tan I go sop in of sgois in the shars My husepd sos to the wek in shops I go to bed at nat my husDd is Mr [xxxxxxx] to little girl is mary are segin en I am a is lid I go sop in at gois in the shop to weall irned of enemo we are eggof Brs to et wekan go ar in the car to sesd vet is somhorse to nad li to go on a riad to the woods at the get  Words"
# Correct the text in parallel
corrected_essay_parallel = process_large_text_parallel(essay_text)
print(corrected_essay_parallel)


I love i my 25 years old. we all the same, a i I was. PJ in the home in the house 3, in my home I get in the mon en I get the key in the mon en I get my head up of wek at sckot My house is wait for ne 20 en I had a tea of mere I at a Sikaimukusto I was up in my tan I go up in of sgois in the park My husepd sos to the wek in shops I go to bed at nat my husband is a [s] to little girl is a are me en I am a is lid I go on in at home in the shop to be kind of enemo we are. But to et a go on in the car to sesd vet ish to nad li to go on a ride to the woods at the get  Words.


In [14]:
doc = nlp(text)


print(doc._.performed_spellCheck) #Should be True
print(doc._.outcome_spellCheck) #Performs the correction


True
When I am 25 chemistry is important to me because it is what I am going to do. (I hope) I would live in a semi detached house with three bed-rooms. I hope to have a family and only two children. I would leave my children in the care of a dedicated child mother while I go to work. I will learn to drive and have a car. (a Estate car if possible) I would come home from work at half past 4 and get tea ready. I would pay the child in 3 a week. (or 3005 because of the metric system) When my children go to school I would help them a little bit (not to much) with they home work at weekends we shall go down to the farm. (on the_ and have lunch. For birthday treats I will let my children pick were they want to go I would let there that ran free and 'n't laugh at them if they believe in tears I will let the pick there own holidays. (as long as there not to be BUT I wouldnot let them run to free. I would care and look after them all they are to old to be looked after when they have a family I